In [124]:
import pandas as pd

In [125]:
import pandas as pd

# 데이터 불러오기
df_good = pd.read_csv('/content/경기도 수원시 착한가격업소 정보_20250318 (1).csv', encoding="cp949")
df_honest = pd.read_csv('/content/경기도 수원시_모범음식점 현황_20240827 (1).csv', encoding="cp949")

In [126]:
# 기타요식업 중 '커피' or '카페' 포함 → '카페'로 수정
condition = (df_good['업종'] == '기타요식업') & (
    df_good['상호명'].str.contains('커피|카페', na=False)
)
df_good.loc[condition, '업종'] = '카페'

# 요식업 관련 업종만 필터링
good_cols = ['한식', '중식', '일식', '양식', '기타요식업']
df_good_merge = df_good[df_good['업종'].isin(good_cols)].copy()

df_honest_merge = df_honest.copy()

In [127]:
# 상호명 기반 업종 간소화
simplify_category = {
    '한식': '한식',
    '분식': '분식',
    '일식': '일식',
    '식육(숯불구이)': '한식',
    '양식': '양식',
    '경양식': '양식',
    '뷔페식': '한식',
    '중국식': '중식',
    '횟집': '일식',
    '외국음식전문점(인도,태국등)': '외국음식전문점',
    '복어취급': '일식',
    '탕류(보신용)': '한식'
}

# 기타로 분류된 값들 하드코딩
shop_to_category = {
    '미소레': '카페/제과',
    '도톰': '횟집',
    '붕붕샐러드': '샐러드',
    '마음(MAUM)': '카페/제과',
    '요고트팜': '카페/제과',
    '지동슈퍼': '카페/제과',
    '까페시모 우만점': '카페/제과',
    '꼼빠도르': '카페/제과',
    '빵마루': '카페/제과',
    '파란풍차제과점': '카페/제과',
    '엘린컴퍼니': '카페/제과',
    '사랑이네 베트남쌀국수': '외국음식전문점',
    '웨이컬리(wakerley)': '카페/제과',
    '명랑꽈배기': '카페/제과',
    '깡우동인계점' : '분식',
    '고베맨숀' : '한식',
}
df_good_merge['업종'] = df_good_merge.apply(
    lambda row: shop_to_category.get(row['상호명'], row['업종']),
    axis=1
)

df_honest_merge['업태'] = df_honest_merge.apply(
    lambda row: shop_to_category.get(row['모범업소명'], row['업태']),
    axis=1
)

In [128]:
# 업종 간소화 매핑 (착한가격업소 + 모범음식점 공통)

df_honest_merge['업태'] = df_honest_merge['업태'].replace(simplify_category)
df_good_merge['업종'] = df_good_merge['업종'].replace(simplify_category)

In [129]:
# 6. 결과 확인

print("착한가격업소 업종:", df_good_merge['업종'].unique())
print("모범음식점 업태:", df_honest_merge['업태'].unique())

print(df_honest_merge[df_honest_merge['업태'] == '기타'])

착한가격업소 업종: ['한식' '양식' '일식' '카페/제과' '샐러드' '중식' '외국음식전문점']
모범음식점 업태: ['한식' '분식' '일식' '양식' '중식' '외국음식전문점']
Empty DataFrame
Columns: [모범업소명, 소재지도로명주소, 소재지지번주소, 위도, 경도, 전화번호, 지정일자, 최신지정일자, 업태, 대표메뉴, 데이터기준일자]
Index: []


In [130]:
df_good_merge['비고'] = '학한가격 업소'
df_honest_merge['비고'] = '모범음식점'

In [131]:
df_honest_merge.columns

Index(['모범업소명', '소재지도로명주소', '소재지지번주소', '위도', '경도', '전화번호', '지정일자', '최신지정일자',
       '업태', '대표메뉴', '데이터기준일자', '비고'],
      dtype='object')

In [133]:
df_good_renamed = df_good_merge.rename(columns={
    '상호명': '업소명',
    '소재지도로명주소': '도로명주소',
    '소재지지번주소': '지번주소',
    '기준일': '기준일자'
})[['업소명', '업종', '도로명주소', '지번주소', '위도', '경도', '전화번호', '기준일자']]

df_honest_renamed = df_honest_merge.rename(columns={
    '모범업소명': '업소명',
    '소재지도로명주소': '도로명주소',
    '소재지지번주소': '지번주소',
    '업태': '업종',
    '데이터기준일자': '기준일자'
})[['업소명', '업종', '도로명주소', '지번주소', '위도', '경도', '전화번호', '기준일자']]

df_merged = pd.concat([df_good_renamed, df_honest_renamed], ignore_index=True)

df_merged.drop_duplicates(subset=['업소명', '위도', '경도'], inplace=True)


df_merged

,업소명,업종,도로명주소,지번주소,위도,경도,전화번호,기준일자
0,예전각전복설렁탕,한식,수원시 장안구 경수대로 816(영화동),경기도 수원시 장안구 영화동 97-29,37.294394,127.017374,031-245-2003,2025-03-18
1,시골밥집,한식,수원시 장안구 파장로76번길 25 (파장동),경기도 수원시 장안구 파장동 579-26,37.307139,126.993342,031-257-6717,2025-03-18
2,풍년식당,한식,수원시 장안구 팔달로 271번길 40 (영화동),경기도 수원시 장안구 영화동 275-7 경북여인숙,37.291154,127.013526,031-243-8237,2025-03-18
3,흥부네정육식당,한식,"수원시 장안구 정자로 146, 한솔빌딩 1층 (정자동)",경기도 수원시 장안구 정자동 454 한솔빌딩,37.303961,126.992355,031-269-0080,2025-03-18
4,진천생고기,한식,수원시 장안구 정조로934번길 25(영화동),경기도 수원시 장안구 영화동 293-38 풍천민물장어,37.291684,127.013994,031-256-3074,2025-03-18
...,...,...,...,...,...,...,...,...
261,호수오면돼지,일식,"경기도 수원시 영통구 센트럴파크로 33, 지하1층 B04호 (하동)",경기도 수원시 영통구 하동 1014-2,37.288563,127.062431,031-217-5353,2024-08-27
262,홍익돈까스(영통망포점),양식,경기도 수원시 영통구 태장로54번길 102 (망포동),경기도 수원시 영통구 망포동 649-16,37.234608,127.051574,031-206-3811,2024-08-27
263,홍천화로구이,한식,"경기도 수원시 영통구 영통로501번길 48-4 (영통동, 951-1)",경기도 수원시 영통구 영통동 951-1,37.265346,127.076612,031-205-5461,2024-08-27
264,화포식당,한식,"경기도 수원시 영통구 청명남로 54, 1-2층 (영통동)",경기도 수원시 영통구 영통동 1038-3,37.254019,127.077092,031-202-6925,2024-08-27


In [134]:
df_merged.to_csv('merged_data.csv', index=False)